Before you turn this problem in, make sure everything runs as expected. 
  1. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and 
  2. Then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below. 

---

# CSCI 3155 : Assignment 5


Topics Covered:
  - Let binding semantics
  - Scopes
  - Function calls

__YOUR NAME HERE__

Note: Please first run the `TEST HELPER` cell that defines the `passed` function below. Failing to run this cell will make it hard for you to check your work.

In [32]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"Tests Passed (1 point)")
    else print(s"Tests Passed ($points points)") 
}

defined function passed

## Problem 1: Multiple Simultaneous Let Bindings (25 points)

In class, we studied let bindings that assigned a "single" identifier to a "single" expression. Here, we will extend lettuce with multiple let bindings at the same time:

## Example 1 
~~~
let (x, y, z) = (10, 25.6, 30.3) in 
   x - y * z
~~~

The program computes `10 - 25.6 * 30.3`.

## Example 2

 In multi-let binding, we treat all the assignments as happening "sequentially". For instance, the program 

~~~
let (x, y, z) = (10, x, y - x) in 
  x - y * z
~~~

should evaluate the same as

~~~
let x = 10 in 
   let y = x in 
     let z = y - x in 
        x - y * z
~~~

I.e, it should evaluate to 10

## Example 3

~~~
let x = 15 in 
  let (x, y, z) = (x*x, -10 *x, -2*x) in 
     x + y + z
~~~

Note that the usage `x*x` refers to the previous let binding that assigned `x` to `15`. However, the let bindings for `y` and `z` refer to the binding where `x` is redefined to be `15 * 15`.


## Grammar of Lettuce

Let us extend a minimalistic subset of Lettuce by adding a `MultiLet` statement as shown below.
$$\newcommand\Expr{\mathbf{Expr}}$$

$$\begin{array}{rcll}
  \Expr & \Rightarrow & \text{Const}(\mathbf{Double}) \\
  & | & \text{Ident}(\mathbf{String}) \\
  & | & \text{Plus}(\mathbf{Expr}, \mathbf{Expr}) \\
  & | & \text{Mult}(\mathbf{Expr}, \mathbf{Expr})\\
  & | & \text{Let}(\mathbf{Ident}, \mathbf{Expr}, \mathbf{Expr}) \\
  & | & \text{MultiLet}(\mathbf{Ident}*, \mathbf{Expr}*, \mathbf{Expr}) & \leftarrow\ \text{ let (x1, .., xn) = (e1, ...,en) in e } \\
  & & & \text{Note: Number of identifiers n must match number of expressions n, or else evaluate to error }\\
  \end{array}$$
  
The scala definitions are given below.

In [33]:
sealed trait Expr
case class Const(d: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr 
case class Let(id: String, e1: Expr, e2: Expr) extends Expr
case class MultiLet(id: List[String], eList: List[Expr], eBody: Expr) extends Expr

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Mult
defined class Let
defined class MultiLet

## Semantics for MultiLet

$$\newcommand\semrule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}(\text{#3})} $$

Let us write down the semantic rules for a multilet statement:

$$\newcommand\eval{\textit{eval}}$$
$$\semrule{ {n \geq 1},\  \eval(\texttt{e1}, \texttt{env})= v_1,\ v_1 \not= \mathbf{error},\ \texttt{newenv} = env \circ \{ \texttt{x1} \mapsto v_1\} }{ \eval( \texttt{MultiLet([x1,..,xn], [e1,...,en], eBody), env}) = \eval(\texttt{MultiLet([x2,..., xn], [e2, ..., en], eBody)}, \texttt{newenv})}{multilet-non-empty}$$

The semantic rule above tells you that
  - If $n \geq 1$, i.e., your multilet statement has one more bindings happening,
  - Evaluate the very first expression `e1`under the environment `env`.
  - Next, if this is evaluated without an error, it tells you to update the map `env` by binding  `x1` to $v_1$, the result of evaluating `e1` to obtain a new environment called `newenv`.
  - Finally, you should evaluate the "remaining" multilet binding that binds `[x2,...,xn]` to `[e2,...,en]`, respectively under the environment `newenv` from previous step. Note that we are providing a recursive definition of the `MultiLet` statement.

We will now provide the last rule for an "empty" multilet statement:

$$\semrule{}{ \eval( \texttt{MultiLet([], [], eBody), env}) = \eval(\texttt{eBody}, env)}{multilet-empty}$$

We will let you fill in the error rules. The interpreter will raise an `IllegalArgumentException` whenever an error is encountered.

### Interpreter for MultiLet Statements

Implement an interpreter for the lettuce language with `multi-let` statements. Your interpreter does not need to "propagate" error: instead you should throw an `IllegalArgumentException` whenever an error is encountered. 

### Style Guide

Use of var/while/for loops in your solution below is disallowed.


In [34]:
sealed trait Value
case class NumValue(f: Double) extends Value
case object Error extends Value /* -- Do not return Error -- simply throw an new IllegalArgumentException whenever you encounter an erroneous case --*/

type Environment = Map[String, Value]

def evalExpr(e: Expr, env: Environment): Value = {
    
    e match {
        case Const(f) => NumValue(f)
        case Ident(x) => { 
            if (env.contains(x)) { 
                env(x)
            } else {
                throw new IllegalArgumentException("Not found identifier")
            }
        }
        case Plus(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            (v1, v2) match {
                case (NumValue(f1), NumValue(f2)) => NumValue(f1 + f2)
                case _ => throw new IllegalArgumentException("plus failed")
            }
        }
        case Mult(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            (v1, v2) match {
                case (NumValue(f1), NumValue(f2)) => NumValue(f1 * f2)
                case _ => throw new IllegalArgumentException("mult failed")
            }
        }
        case Let(x, e1, e2) => {
            // YOUR CODE HERE
           val v1 = evalExpr(e1, env)
           val env2=env++Map(x -> v1) 
           evalExpr(e2,env2)
        }
       case MultiLet(xList, eList, eBody) => {
            // YOUR CODE HERE
           (xList,eList)match
           {
               case(Nil,Nil)=>evalExpr(eBody,env)
               case (x1::restX,e1::restE)=> {
                   val v1= evalExpr(e1,env)
                   val env2=env++Map(x1 -> v1)
                   evalExpr(MultiLet(restX, restE, eBody),env2)
           }
           
            case _ => throw new IllegalArgumentException
            
        }
       }
    }
}

defined trait Value
defined class NumValue
defined object Error
defined type Environment
defined function evalExpr

In [35]:
//BEGIN TEST
/*
 let (x, y) = (10, 20) in 
    let x = y in 
      x +  x * y
*/
val x = Ident("x")
val y = Ident("y")
val let1 = Let("x", y, Plus(x, Mult(x, y)) )
val mlet1 = MultiLet( List("x", "y"), List(Const(10.0), Const(20.0)), let1)
val v = evalExpr(mlet1, Map.empty)
assert(v == NumValue(420.0), s"Test 1 failed expected: NumValue(420.0), obtained $v")

passed(6)
//END TEST

Tests Passed (6 points)

x: Ident = Ident("x")
y: Ident = Ident("y")
let1: Let = Let("x", Ident("y"), Plus(Ident("x"), Mult(Ident("x"), Ident("y"))))
mlet1: MultiLet = MultiLet(
  List("x", "y"),
  List(Const(10.0), Const(20.0)),
  Let("x", Ident("y"), Plus(Ident("x"), Mult(Ident("x"), Ident("y"))))
)
v: Value = NumValue(420.0)

In [36]:
//BEGIN TEST
/*
 let (x, y) = (10, x) in 
    let x = y in 
      x +  x * y
*/
val x = Ident("x")
val y = Ident("y")
val let1 = Let("x", y, Plus(x, Mult(x, y)) )
val mlet1 = MultiLet( List("x", "y"), List(Const(10.0), x), let1)
val v = evalExpr(mlet1, Map.empty)
assert(v == NumValue(110.0), s"Test Failed: expected NumValue(110.0), your code returns $v")


passed(6)
//END TEST

Tests Passed (6 points)

x: Ident = Ident("x")
y: Ident = Ident("y")
let1: Let = Let("x", Ident("y"), Plus(Ident("x"), Mult(Ident("x"), Ident("y"))))
mlet1: MultiLet = MultiLet(
  List("x", "y"),
  List(Const(10.0), Ident("x")),
  Let("x", Ident("y"), Plus(Ident("x"), Mult(Ident("x"), Ident("y"))))
)
v: Value = NumValue(110.0)

In [37]:
//BEGIN TEST
/*
let (x, y, z, w) = (10, 10, 10, 20 ) in 
  let () = () in 
    let w = w in 
       x *( y + w )
*/

val x = Ident("x")
val y = Ident("y")
val z = Ident("z")
val w = Ident("w")
val ten = Const(10.0)
val twenty = Const(20.0)
val innerLet2 = Let("w", w, Mult(x, Plus(y, w)))
val multiLet1 = MultiLet(Nil, Nil, innerLet2)
val e = MultiLet(List("x","y","z","w"), List(ten, ten, ten, twenty), multiLet1)
val v = evalExpr(e, Map.empty)
assert(v == NumValue(300.0), "Test2 Failed -- expected value NumValue(300.0), obtained value $v")

passed(6)
//END TEST

Tests Passed (6 points)

x: Ident = Ident("x")
y: Ident = Ident("y")
z: Ident = Ident("z")
w: Ident = Ident("w")
ten: Const = Const(10.0)
twenty: Const = Const(20.0)
innerLet2: Let = Let(
  "w",
  Ident("w"),
  Mult(Ident("x"), Plus(Ident("y"), Ident("w")))
)
multiLet1: MultiLet = MultiLet(
  List(),
  List(),
  Let("w", Ident("w"), Mult(Ident("x"), Plus(Ident("y"), Ident("w"))))
)
e: MultiLet = MultiLet(
  List("x", "y", "z", "w"),
  List(Const(10.0), Const(10.0), Const(10.0), Const(20.0)),
  MultiLet(
    List(),
    List(),
    Let("w", Ident("w"), Mult(Ident("x"), Plus(Ident("y"), Ident("w"))))
  )
)
v: Value = NumValue(300.0)

In [38]:
//BEGIN TEST
/*
let (x, y, z, w) = (10, x+10, x-y, z+x ) in 
  let () = () in 
    let w = w in 
       x *( y + w )
*/

val x = Ident("x")
val y = Ident("y")
val z = Ident("z")
val w = Ident("w")
val ten = Const(10.0)

def pl(a: Expr,b:Int):Expr = Plus(a, Const(b))

val innerLet2 = Let("w", w, Mult(x, Plus(y, w)))
val multiLet1 = MultiLet(Nil, Nil, innerLet2)
val e = MultiLet(List("x","y","z","w"), List(ten, pl(x,10), Plus(x,Mult(Const(-1.0), y)), Plus(z,x)), multiLet1)
val v = evalExpr(e, Map.empty)
assert(v == NumValue(200.0), s"Test2 Failed -- expected value NumValue(0.0), obtained value $v")

passed(7)
//END TEST

Tests Passed (7 points)

x: Ident = Ident("x")
y: Ident = Ident("y")
z: Ident = Ident("z")
w: Ident = Ident("w")
ten: Const = Const(10.0)
defined function pl
innerLet2: Let = Let(
  "w",
  Ident("w"),
  Mult(Ident("x"), Plus(Ident("y"), Ident("w")))
)
multiLet1: MultiLet = MultiLet(
  List(),
  List(),
  Let("w", Ident("w"), Mult(Ident("x"), Plus(Ident("y"), Ident("w"))))
)
e: MultiLet = MultiLet(
  List("x", "y", "z", "w"),
  List(
    Const(10.0),
    Plus(Ident("x"), Const(10.0)),
    Plus(Ident("x"), Mult(Const(-1.0), Ident("y"))),
    Plus(Ident("z"), Ident("x"))
  ),
  MultiLet(
    List(),
    List(),
    Let("w", Ident("w"), Mult(Ident("x"), Plus(Ident("y"), Ident("w"))))
  )
)
v: Value = NumValue(200.0)

## Problem 2: Tag Usages of Identifiers with Definitions (35 points)

In this problem, we will extend Lettuce by adding user-defined "tags" to the `Let` bindings/function definitions and usages `Ident` in the abstract syntax. The task is to produce a mapping from each ident-tag back to the tag of the let binding that defines the variable, such that scoping becomes explicit. This kind of analysis is called a "use-def" chain and is used in many compilers to optimize the code that is generated (https://en.wikipedia.org/wiki/Use-define_chain).

## Example 1

~~~
let x (@tag1) = 10 in 
   let y (@tag2 ) = ( let x (@tag3) = 10 in x (@use1) + 100 ) in 
      x (@use2) + y (@use3)
~~~

In this example, we add "comment tags" to each let binding (`tag1`, `tag2` and `tag3`) and to each usage (`use1`, `use2`, `use3`). The tags play no role in the program execution other than giving names to refer to the appropriate definitions and usages of identifiers. 

The purpose of this analysis is to produce a Map that maps usage tags to the let binding tags.

~~~
use1 -> tag3
use2 -> tag1
use3 -> tag2
~~~

This denotes that that the usage tagged "use1" corresponds to the let-binding "tag3", "use2" usage corresponds to let binding "tag1" and "use3" usage corresponds to let binding "tag2".

## Example 2

Here is an example with function calls
~~~
let x @tag0 = 100 in 
 let foo @tag3 = function (x @tag1) x @use1 + 100 in 
  let bar @tag4 = function (y @tag2 ) y @use2 + x @use3  in 
    bar@use5 (x@use4)
~~~

To understand the program, simply strip all the tags.

~~~
let x  = 100 in 
 let foo = function (x ) x  + 100 in 
  let bar = function (y ) y  + x   in 
    bar (x)
~~~

We wish to produce the following mapping:

~~~
  use1 -> tag1,
  use2 -> tag2,
  use3 -> tag0,
  use4 -> tag0,
  use5 -> tag4
~~~


### Abstract Syntax

$$\begin{array}{rcll}
  \Expr & \Rightarrow & \text{Const}(\mathbf{Double}) \\
  & | & \text{Ident}(\mathbf{String}, \color{red}{\mathbf{Tag}} ) \\
  & | & \text{Plus}(\mathbf{Expr}, \mathbf{Expr}) \\
  & | & \text{Mult}(\mathbf{Expr}, \mathbf{Expr})\\
  & | & \text{Let}(\mathbf{Ident}, \color{red}{\mathbf{Tag}}, \mathbf{Expr}, \mathbf{Expr}) \\
  & | & \text{FunDef}( \mathbf{Ident}, \color{red}{\mathbf{Tag}}, \mathbf{Expr}) \\
  & | & \text{FunCall}(\mathbf{Expr}, \mathbf{Expr}) \\[5pt]
  \color{red}{\mathbf{Tag}} & \Rightarrow & \mathbf{String} \\
  \end{array}$$
  
We will modify the abstract syntax as shown by the grammar above. Note that we add a new non-terminal called __Tag__ that is simply another name for a scala String datatype. We add these tags to `Ident`, `Let` and `FunDef` which are the places where new identifiers are produced.

Note: 

 - We do not really differentiate use/def tags -- they are all strings that are user defined.
 - All tags must be distinct from one another. This need not be checked.
 

### Part A: Write down Semantic Rules

We will now define the function: `getAllUseDefLinks(e: Expr, env: Map[String, String]): Map[String, String]` that will return a map from the "use tags" strings to the "def tags" strings. 
  - We will use an extra parameter `env` to keep track of a "tag environment", which maps idenfiers to tags. This is similar in one sense to the environment we have seen for `eval` but rather than mapping to a `Value` for each identifier, it maps to a "tag".
  
  


$$\newcommand\semrule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}(\text{#3})} $$
$$\newcommand\getAllUseDefLinks{\textsf{getAllUseDefLinks}}$$

Here is the rule for a let-binding:

$$\semrule{ \getAllUseDefLinks(\texttt{e1}, \text{env}) = \text{map1},\ \text{env}' = \text{env}\circ [id \mapsto tag],\ \getAllUseDefLinks(\texttt{e2}, \text{env}') = \text{map2} }{ \getAllUseDefLinks(\texttt{Let(id, tag, e1, e2)}, \text{env}) = map1 + map2  }{let-binding}$$ 

  - First compute the use-def mappings for the RHS expression `e1`.
  - Next, update the environment to note that identifier `id` is associated with `tag`.
  - Now, compute the mappings for "in" expression `e2` under the new environment.
  - Conjoin the two maps for `e1` and `e2`.

Here is the rule for a function definition:

$$\semrule{ \text{env}' = \text{env} \circ [\texttt{param} \mapsto tag],\ \getAllUseDefLinks(\texttt{eBody}, \text{env}') = \text{map1}}{ \getAllUseDefLinks(\texttt{FunDef(param, tag, eBody)}, \text{env}) = \text{map1} } {fundef}$$ 

Here are two rules for a identifiers:


$$\semrule{ \texttt{id} \in \mathsf{domain}(\text{env})}{ \getAllUseDefLinks(\texttt{Ident(id, usetag)}, \text{env}) = \{ \texttt{usetag} \rightarrow \text{env}(\texttt{id}) \} }{ident-undefined}$$ 

- If the identifier `id` belongs to the domain of the environment, then return a singleton mapping from the use-tag `usetag` associated with this use of `id` to the tag looked up from the environment.


$$\semrule{ \texttt{id} \not\in \mathsf{domain}(\text{env})}{ \getAllUseDefLinks(\texttt{Ident(id, usetag)}, \text{env}) = \{ \texttt{usetag} \rightarrow  \text{"UNDEFINED"} \} }{ident-defined}$$ 

Note that we reserve a special tag name `UNDEFINED` (i.e., the string "UNDEFINED") to denote identifiers that are not defined in the current scope.

- If the identifier `id` does not belong to the domain of the environment, then return a singleton mapping from the use-tag `usetag` associated with this use of `id` to the special tag `UNDEFINED`.


### Part (A) : Complete the rule for Plus (5 points).

$$\semrule{ ???_1}{ \getAllUseDefLinks(\texttt{Plus(e1, e2)}, \text{env}) = ???_1 }{plus}$$ 

Write down what should go in for $???_1$ and $???_2$ below.

 - If there are no antecedents just write "blank". 
 - The empty map is represented as $\{\}$.
 - Concatenation of two maps is `map1` and `map2` is denoted `map1 + map2`.


$$\semrule{getAllUseDefLinks(e1,env)=map1,getAllUseDefLinks(e2,env)=map2 }{ \getAllUseDefLinks(\texttt{Plus(e1, e2)}, \text{env}) = map1+map2 }{plus}$$ 

## Complete the rule for FunCall (5 points)

$$\semrule{ ???_1}{ \getAllUseDefLinks(\texttt{FunCall(e1, e2)}, \text{env}) = ???_1 }{funcall}$$ 

Write down what should go in for $???_1$ and $???_2$ below.

 - If there are no antecedents just write "blank". 
 - The empty map is represented as $\{\}$.
 - Concatenation of two maps is `map1` and `map2` is denoted `map1 + map2`.



$$\semrule{ getAllUseDefLinks(e1,env)=map1,getAllUseDefLinks(e2,env)=map2}{ \getAllUseDefLinks(\texttt{FunCall(e1, e2)}, \text{env}) = map1+map2 }{funcall}$$ 

## Complete the rule for Const (5 points)

$$\semrule{ ???_1}{ \getAllUseDefLinks(\texttt{Const(f)}, \text{env}) = ???_1 }{funcall}$$ 

Write down what should go in for $???_1$ and $???_2$ below.

  - If there are no antecedents just write "blank". 
  - The empty map is represented as $\{\}$.
  - Concatenation of two maps is `map1` and `map2` is denoted `map1 + map2`.

$$\semrule{blank }{ \getAllUseDefLinks(\texttt{Const(f)}, \text{env}) = \{\} }{funcall}$$ 

In [39]:
sealed trait Expr
type Tag = String 
type Identifier = String 

case class Const(d: Double) extends Expr
case class Ident(id:Identifier, tag: Tag) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr 
case class Let(id: Identifier, tag: Tag, e1: Expr, e2: Expr) extends Expr
case class FunDef(param: Identifier, tag: Tag, body:Expr) extends Expr
case class FunCall(e1: Expr, e2:Expr) extends Expr 

// Some helpful functions to write test cases -- you can ignore these.

def x(tag:String):Expr = Ident("x",  tag)
def y(tag:String):Expr = Ident("y",  tag)
def z(tag:String):Expr = Ident("z",  tag)
def w(tag:String):Expr = Ident("w", tag)




defined trait Expr
defined type Tag
defined type Identifier
defined class Const
defined class Ident
defined class Plus
defined class Mult
defined class Let
defined class FunDef
defined class FunCall
defined function x
defined function y
defined function z
defined function w

In [44]:
def getAllUseDefLinks(e: Expr, env: Map[Identifier, Tag]): Map[Tag, Tag] = 

    e match {
        case Const(_) => { Map.empty }
        case Ident(id, usetag) => {
            // YOUR CODE HERE
            if (env.contains(id)) { 
                Map(usetag-> env(id))
            } else {
                Map(usetag->"UNDEFINED")
            }
        }
    
        case Plus(e1, e2) => {
            // YOUR CODE HERE
      
          getAllUseDefLinks(e1,env)++getAllUseDefLinks(e2,env)
            
        }
    
        case Mult(e1, e2) => {
            // YOUR CODE HERE
         getAllUseDefLinks(e1,env)++getAllUseDefLinks(e2,env)

            
           
        }
    
        case Let(x, tag, e1, e2) => {
            // YOUR CODE HERE
            val map1=getAllUseDefLinks(e1,env)
            val env2=env+(x->tag)
            val map2=getAllUseDefLinks(e2,env2)
            map1++map2
        }
    
        case FunDef(x, tag, eBody) => {
            // YOUR CODE HERE
            val env2=env+(x->tag)
            getAllUseDefLinks(eBody,env2)
            //Closure(x, e, env)
        }
    
        case FunCall(e1, e2) => {
            // YOUR CODE HERE
            val map1 = getAllUseDefLinks(e1,env)
            val map2 = getAllUseDefLinks(e2,env)
               map1++map2
        }
}


defined function getAllUseDefLinks

In [45]:
/*
 let y@tag2 = x @use0 in 
    let x @tag1 = 10 in 
       x@use1 + x@use2
*/
val e = Let("y", "tag2", x("use0"), Let ("x", "tag1", Const(10), Plus(x("use1"), x("use2"))))
val map = getAllUseDefLinks(e, Map.empty)

assert (map("use0") == "UNDEFINED", s"Test1 failed: use0 must be associated with UNDEFINED. Your code returns ${map("use0")}")
assert (map("use1") == "tag1", s"Test1 failed: use0 must be associated with UNDEFINED. Your code returns ${map("use1")}")
assert (map("use2") == "tag1", s"Test1 failed: use0 must be associated with UNDEFINED. Your code returns ${map("use2")}")

passed(5)

Tests Passed (5 points)

e: Let = Let(
  "y",
  "tag2",
  Ident("x", "use0"),
  Let("x", "tag1", Const(10.0), Plus(Ident("x", "use1"), Ident("x", "use2")))
)
map: Map[Tag, Tag] = Map(
  "use0" -> "UNDEFINED",
  "use1" -> "tag1",
  "use2" -> "tag1"
)

In [46]:
/* 
let bar @tag2 = function (x @tag3) x @use0 + 10 in 
   let y@tag1 = 10 in 
      x@use1 * y @use2 */

val e = Let("bar", "tag2", FunDef("x", "tag3", Plus(x("use0"), Const(10.0))), 
            Let ("y", "tag1", Const(10), Mult(x("use1"), y("use2"))))
val map = getAllUseDefLinks(e, Map.empty)
assert (map("use0") == "tag3", s"Test1 failed: use0 must be associated with UNDEFINED. Your code returns ${map("use0")}")
assert (map("use1") == "UNDEFINED", s"Test1 failed: use1 must be associated with UNDEFINED. Your code returns ${map("use1")}")
assert (map("use2") == "tag1", s"Test1 failed: use2 must be associated with tag1. Your code returns ${map("use2")}")

passed(5)

Tests Passed (5 points)

e: Let = Let(
  "bar",
  "tag2",
  FunDef("x", "tag3", Plus(Ident("x", "use0"), Const(10.0))),
  Let("y", "tag1", Const(10.0), Mult(Ident("x", "use1"), Ident("y", "use2")))
)
map: Map[Tag, Tag] = Map(
  "use0" -> "tag3",
  "use1" -> "UNDEFINED",
  "use2" -> "tag1"
)

In [47]:
/* 
let x @tag0 = 10 in 
let w@tag1 = function (x @tag2) x @use0 + 10 in 
   let y@tag2 = x@use1 in 
      w@use4 (x@use2 * y @use3) */


val fc:Expr = FunCall( w("use4"), Plus(x("use2"), y("use3")))
val innerLet2:Expr = Let("y", "tag2", x("use1"), fc)
val innerLet1:Expr = Let("w", "tag1", FunDef("x", "tag2", Plus(x("use0"), Const(10))), innerLet2)
val e = Let("x", "tag0", Const(10), innerLet1)


val map = getAllUseDefLinks(e, Map.empty)

assert (map("use0") == "tag2", s"Test1 failed: use0 must be associated with tag2. Your code returns ${map("use0")}")
assert (map("use1") == "tag0", s"Test2 failed: use1 must be associated with tag0. Your code returns ${map("use1")}")
assert (map("use2") == "tag0", s"Test3 failed: use2 must be associated with tag0. Your code returns ${map("use2")}")
assert (map("use3") == "tag2", s"Test4 failed: use2 must be associated with tag2. Your code returns ${map("use2")}")
assert (map("use4") == "tag1", s"Test5 failed: use2 must be associated with tag1. Your code returns ${map("use2")}")


passed(10)

Tests Passed (10 points)

fc: Expr = FunCall(
  Ident("w", "use4"),
  Plus(Ident("x", "use2"), Ident("y", "use3"))
)
innerLet2: Expr = Let(
  "y",
  "tag2",
  Ident("x", "use1"),
  FunCall(Ident("w", "use4"), Plus(Ident("x", "use2"), Ident("y", "use3")))
)
innerLet1: Expr = Let(
  "w",
  "tag1",
  FunDef("x", "tag2", Plus(Ident("x", "use0"), Const(10.0))),
  Let(
    "y",
    "tag2",
    Ident("x", "use1"),
    FunCall(Ident("w", "use4"), Plus(Ident("x", "use2"), Ident("y", "use3")))
  )
)
e: Let = Let(
  "x",
  "tag0",
  Const(10.0),
  Let(
    "w",
    "tag1",
    FunDef("x", "tag2", Plus(Ident("x", "use0"), Const(10.0))),
    Let(
      "y",
      "tag2",
      Ident("x", "use1"),
      FunCall(Ident("w", "use4"), Plus(Ident("x", "use2"), Ident("y", "use3")))
    )
  )
)
map: Map[Tag, Tag] = HashMap(
  "use3" -> "tag2",
  "use4" -> "tag1",
  "use0" -> "tag2",
  "use1" -> "tag0",
  "use2" -> "tag0"
)

## That's All Folks